In [3]:
import numpy as np
import pydot
from IPython.display import display, SVG, clear_output

from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.all import (Parser, StartMeshcat, DiagramBuilder,
                        MeshcatVisualizerCpp, JacobianWrtVariable,
                        MakeRenderEngineVtk, RenderEngineVtkParams)
from pydrake.geometry import (Box, Cylinder)
from pydrake.multibody.tree import (PrismaticJoint, UnitInertia, SpatialInertia, RevoluteJoint, FixedOffsetFrame, WeldJoint)
from manipulation.meshcat_cpp_utils import MeshcatJointSliders
from manipulation.scenarios import AddMultibodyTriad
meshcat = StartMeshcat()

Meshcat is now available at http://localhost:7000


In [41]:
# Build robot model in Drake
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, 0.0)

# parameters
L0 = 1.2 #lenght of link 0
L1 = 0.5
L2 = 0.8
L3 = 0.6

RGBA_Color0 = [0.5, 0.5, 0.5, 0.4]
RGBA_Color1 = [0, 0.5, 0.5, 0.4]
RGBA_Color2 = [0.9, 0, 0, 0.4]
RGBA_Color3 = [0.9, 0.3, 0, 0.4]

my_model_instance = plant.AddModelInstance("my_robot")

#SpatialInertia(mass, reference point (wrt CoM), UnitInertia()): 6x6 matrix
default_inertia = SpatialInertia(1, [0, 0, L0/2], UnitInertia(1, 1, 1))  #This does not matter for kinematics

link_0 = plant.AddRigidBody("link_0", my_model_instance, default_inertia)

link_1 = plant.AddRigidBody("link_1", my_model_instance, default_inertia)

link_2 = plant.AddRigidBody("link_2", my_model_instance, default_inertia)

link_3 = plant.AddRigidBody("link_3", my_model_instance, default_inertia)


# plant.RegisterVisualGeometry(body, 
c =0.05
plant.RegisterVisualGeometry(
    link_0,
    RigidTransform(RollPitchYaw(0, 0, 0), [0, 0, L0/2]),
    Cylinder(c, L0),
    "link_0",
    RGBA_Color0)
plant.RegisterVisualGeometry(
    link_1,
    RigidTransform(RollPitchYaw(0, 0, 0), [0, 0, L1/2]),
    Box(c,c, L1),
    "link_1",
    RGBA_Color1)
plant.RegisterVisualGeometry(
    link_2,
    RigidTransform(RollPitchYaw(0, 0, 0), [0, 0, L2/2]),
    Box(c,c, L2),
    "link_2",
    RGBA_Color2)
plant.RegisterVisualGeometry(
    link_3,
    RigidTransform(RollPitchYaw(0, 0, 0), [0, 0, L3/2]),
    Box(c,c, L3),
    "link_3",
    RGBA_Color3)


Joint1_Frame = plant.AddFrame(FixedOffsetFrame(
    link_0,
    RigidTransform(RollPitchYaw(0, np.pi/2, 0), [0, 0, L0])))

Joint2_Frame = plant.AddFrame(FixedOffsetFrame(
    link_1,
    RigidTransform(RollPitchYaw(0, np.pi/2, 0), [0, 0, L1])))

Joint3_Frame = plant.AddFrame(FixedOffsetFrame(
    link_2,
    RigidTransform(RollPitchYaw(0, -np.pi/2, 0), [0, 0, L2])))

Joint1 = plant.AddJoint(RevoluteJoint(
    name="Joint1", frame_on_parent=Joint1_Frame,
    frame_on_child=link_1.body_frame(), axis=[-1, 0, 0],
    pos_lower_limit=-3.14,
    pos_upper_limit=3.14,
    damping=0.0))
Joint2 = plant.AddJoint(RevoluteJoint(
    name="Joint2", frame_on_parent=Joint2_Frame,
    frame_on_child=link_2.body_frame(), axis=[0, -1, 0],
    pos_lower_limit=-3.14,
    pos_upper_limit=3.14,
    damping=0.0))
Joint3 = plant.AddJoint(RevoluteJoint(
    name="Joint3", frame_on_parent=Joint3_Frame,
    frame_on_child=link_3.body_frame(), axis=[0, 0, 1],
    pos_lower_limit=-3.14,
    pos_upper_limit=3.14,
    damping=0.0))

plant.WeldFrames(
    frame_on_parent_P=plant.world_frame(),
    frame_on_child_C=link_0.body_frame(),
    X_PC=RigidTransform.Identity())


# add frames of interest
G = plant.AddFrame(FixedOffsetFrame('EndEffector',
    link_3,
    RigidTransform(RollPitchYaw(0, -np.pi/2, 0), [0, 0, L3])))

# Draw RGB frames for visualization
for body_name in ["link_0", "link_1", "link_2",'link_3']:
    AddMultibodyTriad(plant.GetFrameByName(body_name), scene_graph, 0.20, 0.008)


In [42]:
# Finalize and visualize
plant.Finalize()
renderer_name = "renderer"
scene_graph.AddRenderer(renderer_name, MakeRenderEngineVtk(RenderEngineVtkParams()))
meshcat.Delete()
meshcat_vis = MeshcatVisualizerCpp.AddToBuilder(builder, scene_graph, meshcat)
diagram = builder.Build()
diagram_context = diagram.CreateDefaultContext()
diagram.Publish(diagram_context)

In [43]:
# Get world frame
theta1= 0
theta2= 0
theta3= 0
plant.SetPositions(plant.GetMyContextFromRoot(diagram_context), my_model_instance, [theta1, theta2,theta3]) # theta1, theta2
diagram.Publish(diagram_context)
plant_context = plant.GetMyMutableContextFromRoot(diagram_context)
X_WG=G.CalcPoseInWorld(plant_context)
print(X_WG)


RigidTransform(
  R=RotationMatrix([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0],
  ]),
  p=[1.1, 0.0, 0.39999999999999997],
)


In [44]:
# now let's verify using PoE 

import modern_robotics as mr

# step 0: compute M 
M = mr.RpToTrans(np.array([[1,0,0],[0,1,0],[0,0,1]]), np.array([L1+L3, 0, L0-L2]))
# step 1: compute all the screw axis
Sbar_3_0=mr.ScrewToAxis(np.array([L1,0,L0-L2]),np.array([0,0,1]),0)
Sbar_2_0=mr.ScrewToAxis(np.array([L1,0,L0]),np.array([0,-1,0]),0)
Sbar_1_0=mr.ScrewToAxis(np.array([0,0,0]),np.array([0,0,1]),0)

def myPoE(theta1,theta2,theta3):
    SbarMatrix_3_0=mr.VecTose3(Sbar_3_0)
    SbarMatrix_2_0=mr.VecTose3(Sbar_2_0)
    SbarMatrix_1_0=mr.VecTose3(Sbar_1_0)
    return mr.MatrixExp6(SbarMatrix_1_0*theta1)@mr.MatrixExp6(SbarMatrix_2_0*theta2)@mr.MatrixExp6(SbarMatrix_3_0*theta3)@M

Tbs = myPoE(theta1,theta2,theta3)
print(Tbs)
print(Sbar_1_0)

[[1.  0.  0.  1.1]
 [0.  1.  0.  0. ]
 [0.  0.  1.  0.4]
 [0.  0.  0.  1. ]]
[0 0 1 0 0 0]


In [45]:
#Blist = np.array([[0,0,1,0,0,0],[0,-1,0,1.2,0,-0.5]]).T
Blist = np.array([Sbar_1_0,Sbar_2_0,Sbar_3_0]).T
my_J = mr.JacobianSpace(Blist, np.array([theta1, theta2,theta3]))
print(my_J)

W = plant.world_frame()
J_WG = plant.CalcJacobianSpatialVelocity(plant_context,JacobianWrtVariable.kV, G, [0,0,0],W,W )
print(J_WG)

J_WG_p = plant.CalcJacobianSpatialVelocity(plant_context,JacobianWrtVariable.kV, G, [1,2,3],W,W )
print(J_WG_p)


[[ 0.   0.   0. ]
 [ 0.  -1.   0. ]
 [ 1.   0.   1. ]
 [ 0.   1.2  0. ]
 [ 0.   0.  -0.5]
 [ 0.  -0.5  0. ]]
[[-6.12323400e-17  0.00000000e+00  1.00000000e+00]
 [ 0.00000000e+00 -1.00000000e+00  0.00000000e+00]
 [ 1.00000000e+00 -0.00000000e+00  6.12323400e-17]
 [ 0.00000000e+00  8.00000000e-01  0.00000000e+00]
 [ 1.10000000e+00  0.00000000e+00 -1.87717473e-17]
 [ 0.00000000e+00  6.00000000e-01  0.00000000e+00]]
[[-6.1232340e-17  0.0000000e+00  1.0000000e+00]
 [ 0.0000000e+00 -1.0000000e+00  0.0000000e+00]
 [ 1.0000000e+00 -0.0000000e+00  6.1232340e-17]
 [-2.0000000e+00 -2.2000000e+00 -1.2246468e-16]
 [ 2.1000000e+00  0.0000000e+00 -3.0000000e+00]
 [-1.2246468e-16  1.6000000e+00  2.0000000e+00]]
